In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train_processed.csv"

# Load the latest version
train_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "buddycyph/train-processed",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", train_df.head())

<ipython-input-31-05ff9d98374a>:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  train_df = kagglehub.load_dataset(


First 5 records:    Score                                               Text  \
0      5  I received this product early from the seller!...   
1      5  *****<br />Numi's Collection Assortment Melang...   
2      5  I was very careful not to overcook this pasta,...   
3      5  Buying this multi-pack I was misled by the pic...   
4      5  These bars are so good! I loved them warmed up...   

                                      Text_Processed  
0  receive product early seller tastey great midd...  
1  br numis collection assortment melange include...  
2  careful overcook pasta make sure take bite eve...  
3  buying multipack mislead picture whole hazel n...  
4  bar good love warm definitely think great snac...  


In [32]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 
import keras

# Adjust scores to be 0-indexed
train_df['Score'] = train_df['Score'] - 1

# Optional: view class distribution
print("Class Distribution:")
print(train_df['Score'].value_counts())

# Text and Labels
texts = train_df['Text_Processed'].values
labels = train_df['Score'].values
y = to_categorical(labels, num_classes=5)

# Tokenization for Word2Vec
tokenized_texts = [word_tokenize(text) for text in texts]

# Train Word2Vec model
w2v_dim = 100
w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=w2v_dim, window=5, min_count=2, workers=4)

# Tokenization for Keras
max_words = 8000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
X_pad = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

# Build Embedding Matrix
vocab_size = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((vocab_size, w2v_dim))

for word, i in word_index.items():
    if i >= max_words:
        continue
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

Class Distribution:
Score
4    197151
3     43876
0     28521
2     23296
1     16287
Name: count, dtype: int64


In [33]:
import keras_tuner as kt

# Define the model-building function
def build_model(hp):
    model = Sequential([
        Embedding(
            input_dim=vocab_size, 
            output_dim=w2v_dim, 
            weights=[embedding_matrix],
            input_length=max_len,
            trainable=True  # Keep Word2Vec embeddings fixed
        ),
        SpatialDropout1D(0.3),
        Bidirectional(LSTM(
            hp.Int('units_1', min_value=32, max_value=128, step=32), 
            return_sequences=True, 
            kernel_regularizer=l2(0.001)
        )),
        BatchNormalization(),
        Bidirectional(LSTM(
            hp.Int('units_2', min_value=32, max_value=128, step=32),
            kernel_regularizer=l2(0.001)
        )),
        Dropout(hp.Float('dropout', min_value=0.4, max_value=0.6, step=0.05)),
        Dense(hp.Int('dense_units', min_value=16, max_value=64, step=16), activation='relu', kernel_regularizer=l2(0.001)),
        Dense(5, activation='softmax')
    ])
    
    # Compile the model with a learning rate that is also tunable
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=hp.Float('lr', min_value=1e-6, max_value=1e-2, sampling='log')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Instantiate the tuner
tuner = kt.RandomSearch(
    build_model, 
    objective='val_accuracy', 
    max_trials=5,  # Number of trials to run
    executions_per_trial=1,  # Number of models to train per trial
    directory='tuner_results',
    project_name='hyperparameter_tuning'
)

# Perform the search for the best hyperparameters
tuner.search(X_train, y_train, 
             epochs=10, 
             batch_size=64, 
             validation_data=(X_test, y_test), 
             callbacks=[early_stopping, lr_scheduler])

# Get the best model and hyperparameters
best_model = tuner.get_best_models(1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

# Print best hyperparameters
print("Best Hyperparameters:")
print(best_hyperparameters.values)

# Evaluate the best model
loss, accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Trial 5 Complete [00h 11m 54s]
val_accuracy: 0.7257508635520935

Best val_accuracy So Far: 0.7257508635520935
Total elapsed time: 00h 58m 04s


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


TypeError: BaseTuner.get_best_hyperparameters() got an unexpected keyword argument 'num_models'

In [8]:
file_path = "sampleSubmission.csv"

# Load the latest version
sample_submission = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "buddycyph/samplesubmission",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

<ipython-input-8-a7175d73b4df>:4: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  sample_submission = kagglehub.load_dataset(


In [10]:
file_path = "test.csv"

# Load the latest version
test_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "buddycyph/testdataset",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

<ipython-input-10-45bd6c1e7e78>:4: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  test_df = kagglehub.load_dataset(


In [12]:
test_texts = test_df['Text'].values
test_sequences = tokenizer.texts_to_sequences(test_texts)
X_test_pad = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

In [13]:
y_pred_proba = model.predict(X_test_pad)
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred = y_pred + 1 

3740/3740 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step


In [14]:
sample_submission['Score'] = y_pred

# 7. Save to CSV for Kaggle submission
sample_submission.to_csv("john_cena.csv", index=False)